# CamHD Video File to Azure Blob with Dask

In [ ]:
## inspect your database
import pandas as pd
# this option will show the full filename
pd.set_option('display.max_colwidth', -1)
dbcamhd = pd.read_json('/home/jovyan/floc_data/dbcamhd.json', orient="records", lines=True).sort_values(by=['timestamp'])
dbcamhd.tail()

In [ ]:
# lets make a small set to test with
c = []
url = []
for i, row in dbcamhd[14400:14407].iterrows():
    file = row.filename
    c.append(file.split('/')[-1])
    url.append(file)

In [ ]:
# put the data in a dataframe
short = pd.DataFrame({'file': c, 'url': url})

In [ ]:
short

#### Start a Dask cluster

In [ ]:
#get yer dask going
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
from dask.distributed import Client
import dask
client = Client(cluster)
client

In [ ]:
from dask import delayed

## Dask Dl Function

In [ ]:
#Here we will try to DL with Dask
### the print command did not work but I would like it to!
import requests
import os
import sys

@delayed
def delayed_dl_file(download_dict):
    ## we use k and v here for key and value from our dictionary (download_dict)
    for k,v in download_dict.items():
        # NOTE the stream=True parameter
        r = requests.get(k, stream=True)
        with open(os.path.join(paths, v), 'wb') as f:
            #Use chunks to download the file in "chunks" instead of loading the entire file at once.
            for chunk in r.iter_content(chunk_size=5024): 
               ## 200 is the code for HTTP status = "OK"
                if r.status_code == 200:
                    f.write(chunk)
        print("File" , k , "created in", paths)

In [ ]:
def write_pickle(containers, account_key, expiration, filename, overwrite=False, include_account_key=False):
    if type(containers) == list:
        keys = dict()
        for container in containers:
            container_sas = gen_token(container, account_key, expiration)
            keys[container] = container_sas
        if include_account_key == True:
            keys['ooiopendata'] = account_key
        pickle_path = os.path.abspath(filename)
        if os.path.isfile(pickle_path) == False:
            with open(pickle_path, 'wb') as f:
                pickle.dump(keys, f, protocol=0)
        elif overwrite == True:
            with open(pickle_path, 'wb') as f:
                pickle.dump(keys, f, protocol=0)
        else:
            raise Exception('File exists.')
    else:
        raise TypeError('The \'containers\' argument must be a list.')

In [ ]:
local_filename = []
for i in urls:
    local_filename.append(i.split('/')[-1])

## Upload file from URL to Microsoft Azure Blob Storage

In [ ]:
# # import modules
import azure.storage.blob as ASB
import azure.storage.common
import yaml

In [ ]:
# load Azure storage account credentials
with open('/home/jovyan/.azure_credentials_ooitest.yaml', 'r') as f:
    credentials = yaml.load(f)
azure_storage_account_name = credentials['azure_storage_account_name']
azure_storage_account_key = credentials['azure_storage_account_key']

In [ ]:
block_blob_service = ASB.BlockBlobService(azure_storage_account_name, azure_storage_account_key)
container_name ='movtest'

In [ ]:
block_blob_service.create_container(container_name)

In [ ]:
# define function to copy from remote url to blob storage
def run_sample():
       block_blob_service.copy_blob(container_name,'rawdata','https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/09/CAMHDA301-20190709T031500.mov')



In [ ]:
 # run function
if __name__ == '__main__':
     run_sample()

## Check size of containters & blobs to see if it worked

In [ ]:
containers = block_blob_service.list_containers()
for c in containers:
        contName = c.name
        generator = block_blob_service.list_blobs(contName)
        contSize = 0
        contNum = 0
        for blob in generator: 
            contSize += blob.properties.content_length
            contNum += 1
        print(contName + " : " + str(contSize/1000000) + " MB" + " in " + str(contNum) + " blobs")

## Clean up your space before you go home

### clean up blobs

In [ ]:
blob_list = block_blob_service.list_blobs(container_name)
for blob in blob_list: 
    blob_name = blob.name
    print(blob_name)

In [ ]:
# # delete all blobs in container
# blob = block_blob_service.list_blobs(container_name)
# #for a in blob:
#        #blob_service.delete_blob(container_name = container_name, blob_name = a.name)

### clean up containers

In [ ]:
# # #set container to delete
# container_name = 'mp4test'

In [ ]:
# # Clean up resources. This includes the container and the temp files.
block_blob_service.delete_container(container_name)
# # os.remove(full_path_to_file)
# # os.remove(full_path_to_file2)